In [1]:
#1.Install the required packages
!pip -q install langchain langchain-community
!pip -q install pypdf
!pip -q install sentence_transformers
!pip install openai
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.6 MB/s eta 0:00:00


In [2]:
!pip install tokenizers
!pip install faiss-cpu #Faiss vector database
!pip -q install unstructured #for loading the website or extract the data from website

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 9.5 MB/s eta 0:00:00


In [3]:
!pip install numpy==1.24.4
!pip install nltk==3.9.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 48.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
faiss-cpu 1.9.0 requires numpy<3.0,>=1.25.0, but you have numpy 1.24.4 which is incompatible.


In [2]:
#Import the neccessary libraries
import sys
import os
import torch
import textwrap
from langchain.document_loaders import UnstructuredURLLoader #for loading the html doc from the website
from langchain.text_splitter import CharacterTextSplitter #chunking
from langchain.embeddings import OpenAIEmbeddings #Convert data into vectors
from langchain.chat_models import ChatOpenAI #Openai LLM
from langchain.vectorstores import FAISS #Vectordb
from langchain.chains import RetrievalQAWithSourcesChain #For creating the chain



In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
#3.Setting up the environment
os.environ['OPENAI_API_KEY'] = "**********************Your Openai API Key*****************************"


In [4]:
#4.Pass the url's and extract the data from these URL's
URLs=[
    'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb',
    'https://www.mosaicml.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models',
    'https://lmsys.org/blog/2023-03-30-vicuna/'

]

In [6]:
#3.Extract the documents
loaders = UnstructuredURLLoader(urls=URLs)
data = loaders.load()

In [ ]:
data

In [ ]:
len(data)

In [ ]:
#4.Split text into chunks
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [ ]:
text_chunks=text_splitter.split_documents(data)

In [ ]:
len(text_chunks)

In [ ]:
text_chunks[0]

In [ ]:
#5.Downlaod the OpenAI Embeddings or Huggingface embeddings
embeddings = OpenAIEmbeddings()

In [ ]:
embeddings

In [ ]:
query_result = embeddings.embed_query("Hello world")
len(query_result)

In [ ]:
#6.Convert the Text Chunks into Embeddings and Create a Knowledge Base
vectorstore=FAISS.from_documents(text_chunks, embeddings)

In [ ]:
#7.Create a Large Language Model (LLM) Wrapper
llm=ChatOpenAI()

In [ ]:
llm

In [ ]:
llm.predict("Please provide a concise summary of the Book Harry Potter")

In [ ]:
#8.Initialize the Retrieval QA with Source Chain
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorstore.as_retriever())

In [ ]:
result=chain({"question": "How good is Vicuna?"}, return_only_outputs=True)

In [ ]:
result['answer']

In [ ]:
wrapped_text = textwrap.fill(result['answer'], width=500)

In [ ]:
wrapped_text

In [ ]:
result=chain({"question": "What is is stableLM?"}, return_only_outputs=True)

In [ ]:
result['answer']

In [ ]:
wrapped_text = textwrap.fill(result['answer'], width=500)

In [ ]:
wrapped_text

In [ ]:
result=chain({"question": "Can you please share some details about MPT-7b Model"}, return_only_outputs=True)

In [ ]:
result['answer']

In [ ]:
wrapped_text = textwrap.fill(result['answer'], width=100)

In [ ]:
wrapped_text

In [ ]:
while True:
  query=input(f"Prompt: ")
  if query == 'exit':
    print('Exiting')
    sys.exit()
  if query =='':
    continue
  result=chain({'question':query})
  print(f"Answer: " +result["answer"])